In [2]:
import pandas as pd
import pyomo.environ as pe


In [17]:
trained = pd.read_csv('data/train.csv')

In [33]:
trained.apply(lambda x : '_'.join([str(x["galactic year"]), str(x["galaxy"])]), axis=1).to_list()

',
 '991020_Andromeda IX',
 '991020_NGC 3109',
 '991020_Andromeda XXIII',
 '991020_Andromeda XXV',
 '991020_Boötes II',
 '991020_Cassiopeia II (Andromeda XXX)',
 '991020_IC 3104',
 '991020_DDO 99 (UGC 6817)',
 '991020_KUG 1210+301B (KK98 127)',
 '991020_KKR 03 (KK98 230)',
 '991020_Aquarius II',
 '991020_Grus I',
 '991020_Pisces V (Andromeda XVI)',
 '991020_MB 3',
 '991020_UGC 8508 (I Zw 060)',
 '991020_NGC 147 (DDO 3)',
 '991020_Sextans B (UGC 5373)',
 '991020_NGC 3741',
 '991020_Leo A (Leo III, DDO 69)',
 '991020_NGC 4214 (UGC 7278)',
 '991020_Phoenix II',
 '991020_Pisces Dwarf',
 '991020_GR 8 (DDO 155)',
 '991020_Sagittarius Dwarf Irregular Galaxy (SagDIG)',
 '991020_Pisces I',
 '991020_UGC 9128 (DDO 187)',
 '991020_Hydrus I',
 '991020_UGCA 15 (DDO 6)',
 '991020_HIPASS J1247-77',
 '991020_Andromeda V',
 '991020_Ursa Major II Dwarf',
 '991020_Fornax Dwarf (E356-G04)',
 '991020_Sculptor Dwarf (E351-G30)',
 '991020_KK98 35',
 '991020_UGCA 86',
 '991020_Canes Venatici I Dwarf',
 '991020

In [35]:
class galaxy_optim:

    def __init__(self, ml_model_output):
        
        self.m = pe.ConcreteModel()

        # Множество галактик
        self.m.galaxy = pe.Set(initialize = ml_model_output.apply(lambda x : '_'.join([str(x["galactic year"]), str(x["galaxy"])]), axis=1).to_list())

        print(self.m.galaxy)


        # Максимальная суммарная энергия
        self.m.max_sum_energy = pe.Param()
        # Максимальная энергия в галактике
        self.m.max_energy = pe.Param()
        # Индекс благополучия(приходит из МО задачки)
        self.m.index_ = pe.Param(self.m.galaxy)
        # bool True, если existence_expectancy меньше заданного порога
        self.m.low_existence_expectancy = pe.Param(self.m.galaxy)
        # Минимальный процент энергии, выделенной на планеты с маленькой продолжительностью жизни
        self.m.min_low_index_percent_allocation = pe.Param()
        # Потенциал роста, рассчитывается по форме -np.log(Index+0.01)+3
        self.m.potential_encrease = pe.Param(self.m.galaxy, lambda  g : (pe.log(self.m.index_[g] + 0.01) + 3) )


        #Параметры управления
        self.m.energy = pe.Var(self.m.galaxy, domain=pe.NonNegativeReals)



        #Огранияения
        # Likely increase in the Index = extra energy * Potential for increase in the Index **2 / 1000
        self.m.likely_index_increase = pe.Param(self.m.galaxy, lambda  g: (self.m.energy * (self.m.potential_encrease ** 2)) / 1000 )

        #Constraints
        # in total there are 50000 zillion DSML available for allocation
        self.m.max_sum_energy_constraint = pe.Constraint(sum(self.m.existence_expectancy_index[g]* self.m.energy[g] for g in self.m.galaxy) <= self.m.max_sum_energy)

        # no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML\
        self.m.max_energy = pe.Constraint(self.m.galaxy, rule = lambda  g: self.m.energy[g] <= self.m.max_energy)

        # galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available
        self.m.low_index_constraint = pe.Constraint(rule = lambda : sum(self.m.low_existence_expectancy[g] * self.m.energy[g] for g in self.m.galaxy) >= sum(self.m.energy[g] for g in self.m.galaxy) * self.m.min_low_index_percent_allocation)

        #Objective function
        self.m.OBJ = pe.Objective(self.m.galaxy, rule = lambda  g : self.m.likely_index_increase[g], sense = maximize )

        # def ax_constraint_rule(self.m, i):
        #     # return the expression for the constraint for i
        #     return sum(self.m.a[i,j] * self.m.x[j] for j in self.m.J) >= self.m.b[i]

        # # the next line creates one constraint for each member of the set self.m.I
        # self.m.AxbConstraint = Constraint(self.m.I, rule=ax_constraint_rule)


        # # the next line creates one constraint for each member of the set self.m.I
        # self.m.AxbConstraint = Constraint(self.m.I, rule=ax_constraint_rule)

optim = galaxy_optim(trained)

galaxy
ERROR: Constructing component 'potential_encrease_index_1' from data=None
    failed: KeyError: "Index 'unknown' is not valid for indexed component
    'index_'"


KeyError: "Index 'unknown' is not valid for indexed component 'index_'"